This is a script that scrapes global revenue figures from Microsoft Investor Relations (NOTE 17 — SEGMENT INFORMATION AND GEOGRAPHIC DATA; Revenue from external customers, classified by significant product and service offerings). One example from these webpages can be found at https://www.microsoft.com/en-us/Investor/earnings/FY-2021-Q1/IRFinancialStatementsPopups?tag=us-gaap:SegmentReportingDisclosureTextBlock&title=More%20Personal%20Computing.

The following block loads all necessary packages and libraries required by the script. The block needs to be run every time the code is used to scrape data. If required packages are not installed, running the code will throw an error. Refer to Statistics Canada's instructions for installing and requesting packages on your Net B VDI. If Python is not yet installed on your system, you will need to submit an SRM for access.

In [7]:
# the Python Requests package will allow us to send HTTP requests to get HTML files
import requests

# the GET method indicates that you’re trying to get or retrieve data from a specified resource. 
# to make a GET request, invoke requests.get()
from requests import get

# Beautiful Soup is a Python library for pulling data out of HTML and XML files
from bs4 import BeautifulSoup

# pandas is a Python data analysis library
import pandas as pd

# NumPy is a Python library used for working with large, multi-dimensional arrays and matrices
import numpy as np

# the time module in Python has a function sleep() that you can use to suspend execution of the calling thread 
from time import sleep

# The randint() method returns a pseudo-random integer number 
from random import randint

The following block loads the initial Excel file. I created an original file which contains financial data from the 2017 fiscal year, not available through scraping by the below method. The block needs to be run every time the code is opened and used.

In [8]:
# reads the original Excel file I made with 2017 data as a Pandas dataframe
xl_file = pd.ExcelFile('Microsoft.xlsx')

# create a dictionary with sheet names as keys and dataframes corresponding to each sheet as values
# in this case, only a single sheet exists
dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}

# storing the relevant sheet to the name microsoft_data
microsoft_data = dfs.get('Microsoft')

# converting the sheet to another dictionary, with 
my_dict = microsoft_data.to_dict('list')

my_dict

{'Quarter': ['FY2017Q1',
  'FY2017Q1',
  'FY2017Q1',
  'FY2017Q1',
  'FY2017Q1',
  'FY2017Q1',
  'FY2017Q1',
  'FY2017Q1',
  'FY2017Q1',
  'FY2017Q1',
  'FY2017Q2',
  'FY2017Q2',
  'FY2017Q2',
  'FY2017Q2',
  'FY2017Q2',
  'FY2017Q2',
  'FY2017Q2',
  'FY2017Q2',
  'FY2017Q2',
  'FY2017Q2',
  'FY2017Q3',
  'FY2017Q3',
  'FY2017Q3',
  'FY2017Q3',
  'FY2017Q3',
  'FY2017Q3',
  'FY2017Q3',
  'FY2017Q3',
  'FY2017Q3',
  'FY2017Q3',
  'FY2017Q4',
  'FY2017Q4',
  'FY2017Q4',
  'FY2017Q4',
  'FY2017Q4',
  'FY2017Q4',
  'FY2017Q4',
  'FY2017Q4',
  'FY2017Q4',
  'FY2017Q4'],
 'Segment': ['Office products and cloud services',
  'Server products and cloud services',
  'Windows',
  'Gaming',
  'Search advertising',
  'Enterprise Services',
  'Devices',
  'LinkedIn',
  'Other',
  'Total',
  'Office products and cloud services',
  'Server products and cloud services',
  'Windows',
  'Gaming',
  'Search advertising',
  'Enterprise Services',
  'Devices',
  'LinkedIn',
  'Other',
  'Total',
  'Office p

The following block populates the existing Excel file with segment revenues from FYQ1 2018 to FYQ4 2021. However, as of June 17, 2021, Q4 2021 figures are not available on the Microsoft website. Q4 data can be gathered directly once available. This code will duplicate rows if it is run twice. It does not need to be run again unless you wish to add several quarters of financial information at once, in which case you alter the range of years from which data will be required.

In [9]:
# using the NumPy arange() function, we can create vectors that return evenly spaced values within a given interval.
# years is a vector containing the range of years from which data will be scraped (in this case, 2018 to 2021) and should be altered for the desired range
# quarters is the vector [1 2 3 4]
years = np.arange(2018, 2022, 1) 
quarters = np.arange(1, 5, 1)

# iterate through each quarter of each year
for year in years:
    for quarter in quarters:
        
        # specify the URL for each reference date
        url = 'https://www.microsoft.com/en-us/Investor/earnings/FY-' + str(year) + '-Q' + str(quarter) + '/IRFinancialStatementsPopups?tag=us-gaap:SegmentReportingDisclosureTextBlock&title=More%20Personal%20Computing'
        
        # method we use to grab the contents of the URL
        results = requests.get(url)
        
        # soup is the variable we create to assign the method BeatifulSoup
        # The BeautifulSoup library specifies a desired format of results using the HTML parser
        # This allows Python to read the components of the page rather than treating it as one long string
        soup = BeautifulSoup(results.text, "html.parser")
        
        # the necessary financial data are stored in 
        alltables = soup.find_all('table', attrs={'style' : ["margin:auto;border-collapse:collapse; width:100%;", "border-collapse:collapse; width:99.86%;", "border-collapse:collapse; width:100%;"]})
        
        # avoid overloading the website being scraped by reducing the crawl rate
        sleep(randint(2,10))
        
        for table in alltables:
            # only one table contains the necessary segment revenue
            # as the HTML is 
            if ('Office products and cloud services') in str(table):
                rows = table.find_all('tr')

                for row in rows:
                    
                    # some tables have multiple columns of revenue values. We are only interested in the first column
                    # values_list is an empty list created for each row of the table 
                    # it will store all revenue numbers from each row, so the value in the first column will be the first element
                    # this is also a hard-coded solution to distinguishing from numbers in the first column and numbers in other columns
                    values_list = []
                    
                    # the process of finding revenue values on the page is hard-coded for the specific HTML of the Microsoft website, as we look for data based on HTML styles
                    # styles are subject to change in future quarters, so it is vital to reevaluate the data source website when scraping results are unexpected
                    # segment category names in the table are not right-justified                    
                    # The code finds segment names and adds them as dataset rows
                    names = row.find_all('p', style=lambda value: value and ('text-align:justify;' in value or 'text-align:left;' in value) and 'font-size:10pt;' in value)
                    for name in names:
                        date = 'FY' + str(year) + 'Q' + str(quarter)
                        
                        print(date)
                        # adding the date from which the data is scraped from to "Quarter"
                        my_dict['Quarter'].append(date)

                        print(name.text)
                        # adding the name of each segment category to "Segment"
                        my_dict['Segment'].append(name.text)
                    
                    # values corresponding to each segment in the table are right-justified
                    # the find_all command will find all text with the stated properties
                    values = row.find_all('p', style=lambda value: value and 'text-align:right;' in value and 'font-size:10pt;' in value)
                    for value in values:    
                        values_list.append(value.text)
                    
                    # don't count empty strings
                    if len(values_list) > 0:
                        # as previously mentioned, the first item of the list will correspond to the first column
                        # adding the revenue in each row to "Revenue"
                        print(values_list[0]) 
                        my_dict['Revenue'].append(value.text)


# convert the dictionary back to a dataframe
df = pd.DataFrame.from_dict(my_dict)

# the dataframe is saved in a new CSV file
df.to_csv('Microsoft.csv', index = False)

FY2018Q1
Office products and cloud services
6,575
FY2018Q1
Server products and cloud services
5,496
FY2018Q1
Windows
4,643
FY2018Q1
Gaming
1,896
FY2018Q1
Search advertising
1,639
FY2018Q1
Enterprise Services
1,371
FY2018Q1
Devices
1,154
FY2018Q1
LinkedIn
1,148
FY2018Q1
Other 
616
FY2018Q1
Total 
24,538
FY2018Q2
Office products and cloud services
7,075
FY2018Q2
Server products and cloud services
6,299
FY2018Q2
Windows
4,839
FY2018Q2
Gaming
3,920
FY2018Q2
Search advertising
1,820
FY2018Q2
Enterprise Services
1,435
FY2018Q2
Devices
1,478
FY2018Q2
LinkedIn
1,312
FY2018Q2
Other 
740
FY2018Q2
Total
28,918
FY2018Q3
Office products and cloud services
7,088
FY2018Q3
Server products and cloud services
6,343
FY2018Q3
Windows
4,612
FY2018Q3
Gaming
2,251
FY2018Q3
Search advertising
1,784
FY2018Q3
Enterprise Services
1,489
FY2018Q3
Devices
1,219
FY2018Q3
LinkedIn
1,335
FY2018Q3
Other 
698
FY2018Q3
Total
26,819
FY2018Q4
Office products and cloud services
28,316
FY2018Q4
Server products and cloud serv

In [10]:
df

,Quarter,Segment,Revenue
0,FY2017Q1,Office products and cloud services,5982
1,FY2017Q1,Server products and cloud services,4689
2,FY2017Q1,Windows,4643
3,FY2017Q1,Gaming,1885
4,FY2017Q1,Search advertising,1429
...,...,...,...
185,FY2021Q3,Search advertising,"6,140"
186,FY2021Q3,Devices,"4,662"
187,FY2021Q3,Enterprise Services,"4,790"
188,FY2021Q3,Other,"2,775"


The following block adds additional rows by individual quarter and can be used to update the existing file. Running this block will append the reference quarter values to the value vectors for the selected period. This is simply the previous HTML conversion method without a nested loop to accomodate for multiple files.

In [11]:
# year controls the reference year that you wish to scrape
# quarter controls the reference quarter that you wish to scrape
year = 2022
quarter = 1

# specify the URL for the specific reference period
url = 'https://www.microsoft.com/en-us/Investor/earnings/FY-' + str(year) + '-Q' + str(quarter) + '/IRFinancialStatementsPopups?tag=us-gaap:SegmentReportingDisclosureTextBlock&title=More%20Personal%20Computing'

# method we use to grab the contents of the URL
results = requests.get(url)

# soup is the variable we create to assign the method BeatifulSoup
# The BeautifulSoup library specifies a desired format of results using the HTML parser
# This allows Python to read the components of the page rather than treating it as one long string
soup = BeautifulSoup(results.text, "html.parser")

alltables = soup.find_all('table', attrs={'style' : ["margin:auto;border-collapse:collapse; width:100%;", "border-collapse:collapse; width:99.86%;", "border-collapse:collapse; width:100%;"]})

for table in alltables:
    if ('Office products and cloud services') in str(table):
        rows = table.find_all('tr')
        
        for row in rows:

            # some tables have multiple columns of revenue values. We are only interested in the first column
            # values_list is an empty list created for each row of the table 
            # it will store all revenue numbers from each row, so the value in the first column will be the first element
            # this is also a hard-coded solution to distinguishing from numbers in the first column and numbers in other columns
            values_list = []
            
            # the process of finding revenue values on the page is hard-coded for the specific HTML of the Microsoft website, as we look for data based on HTML styles
            # styles are subject to change in future quarters, so it is vital to reevaluate the data source website when scraping results are unexpected    
            # segment category names in the table are not right-justified
            names = row.find_all('p', style=lambda value: value and ('text-align:justify;' in value or 'text-align:left;' in value) and 'font-size:10pt;' in value)
            for name in names:
                date = 'FY' + str(year) + 'Q' + str(quarter)
                
                print(date)
                # adding the date from which the data is scraped from to the "Quarter" value vector
                my_dict['Quarter'].append(date)
        
                print(name.text)
                # adding the name of each segment category to the "Segment" value vector
                my_dict['Segment'].append(name.text)
            
            # values corresponding to each segment in the table are right-justified
            # the find_all command will find all text with the stated properties, so in tables with multiple columns 
            values = row.find_all('p', style=lambda value: value and 'text-align:right;' in value and 'font-size:10pt;' in value)
            for value in values:    
                values_list.append(value.text)
            
            # don't count empty strings
            if len(values_list) > 0:
                
                # as previously mentioned, the first item of the list will correspond to the first column
                # adding the revenue in each row to "Revenue"
                print(values_list[0]) 
                my_dict['Revenue'].append(value.text)

# convert the dictionary back to a dataframe
df = pd.DataFrame.from_dict(my_dict)

# save the modified file into CSV form to load into R

df.to_csv('Microsoft.csv', index = False)

In [12]:
df

,Quarter,Segment,Revenue
0,FY2017Q1,Office products and cloud services,5982
1,FY2017Q1,Server products and cloud services,4689
2,FY2017Q1,Windows,4643
3,FY2017Q1,Gaming,1885
4,FY2017Q1,Search advertising,1429
...,...,...,...
185,FY2021Q3,Search advertising,"6,140"
186,FY2021Q3,Devices,"4,662"
187,FY2021Q3,Enterprise Services,"4,790"
188,FY2021Q3,Other,"2,775"


The data will need to be cleaned before any analysis takes place. Specifically, scraping the relevant table on Microsoft's website will include the revenue total, which is not necessary for analysis. Another issue arises with Q4 values, which are not reported individually. Q4 revenues represent a total of the full fiscal year. To get the value corresponding to the three months belonging to Q4, the value must be subtracted from the sum of the three previous quarters.